In [1]:
from hydroai.gap_prediction import predict_station_gaps

In [2]:
station_file = "./data/lib/station_11.0_cleaned.csv"

reall_combined_dfs, val_full, metrics_gaps, real_predictionst = predict_station_gaps(
    station_file, model_type="rf"
)

In [4]:
ret.keys()

AttributeError: 'tuple' object has no attribute 'keys'